In [14]:
import pandas as pd
import numpy as np
from collections import Counter
from sklearn.model_selection import cross_val_score

df = pd.read_csv('situacao_do_cliente.csv')

X_df = df[['recencia','frequencia','semanas_de_inscricao']]
Y_df = df['situacao']

Xdummies_df = pd.get_dummies(X_df)
Ydummies_df = Y_df

X = Xdummies_df.values
Y = Ydummies_df.values

porcentagem_treino = 0.8

tamanho_treino = int(porcentagem_treino * len(Y))
#tamanho_validacao = len(Y) - tamanho_treino

treino_dados = X[:tamanho_treino]
treino_marcacoes = Y[:tamanho_treino]

validacao_dados = X[tamanho_treino:]
validacao_marcacoes = Y[tamanho_treino:]

resultados = {}

# treinando com OneVsRest
from sklearn.multiclass import OneVsRestClassifier
from sklearn.svm import LinearSVC
modeloOneVsRest = OneVsRestClassifier(LinearSVC(random_state = 0, tol = 1))
resultadoOneVsRest = fit_and_predict("OneVsRest", modeloOneVsRest, treino_dados, treino_marcacoes)
resultados[resultadoOneVsRest] = modeloOneVsRest

# treinando com OneVsOne
from sklearn.multiclass import OneVsOneClassifier
modeloOneVsOne = OneVsOneClassifier(LinearSVC(random_state = 0, tol = 1))
resultadoOneVsOne = fit_and_predict("OneVsOne", modeloOneVsOne, treino_dados, treino_marcacoes)
resultados[resultadoOneVsOne] = modeloOneVsOne

# treinando com MultinomialNB
from sklearn.naive_bayes import MultinomialNB
modeloMultinomial = MultinomialNB()
resultadoMultinomial = fit_and_predict("MultinomialNB", modeloMultinomial, treino_dados, treino_marcacoes)
resultados[resultadoMultinomial] = modeloMultinomial

# treinando com AdaBoost
from sklearn.ensemble import AdaBoostClassifier
modeloAdaBoost = AdaBoostClassifier()
resultadoAdaBoost = fit_and_predict("AdaBoost", modeloAdaBoost, treino_dados, treino_marcacoes)
resultados[resultadoAdaBoost] = modeloAdaBoost

maximo = max(resultados)
vencedor = resultados[maximo]

vencedor.fit(treino_dados, treino_marcacoes)

teste_real(vencedor, validacao_dados, validacao_marcacoes)

acerto_base = max(Counter(validacao_marcacoes).values())
taxa_acerto_base = 100.0 * acerto_base / len(validacao_marcacoes)
print("Taxa de acerto base: %f" % taxa_acerto_base)

total_elementos = len(validacao_dados)
print("Total de teste: %d" % total_elementos)

Taxa de acerto do OneVsRest: 0.9277777777777778
Taxa de acerto do OneVsOne: 0.95
Taxa de acerto do MultinomialNB: 0.8277777777777778
Taxa de acerto do AdaBoost: 0.7611111111111111
Taxa de acerto do vencedor entre os dois algoritmos no mundo real: 97.77777777777777
Taxa de acerto base: 75.555556
Total de teste: 45


In [8]:
def fit_and_predict(nome, modelo, treino_dados, treino_marcacoes):
    k = 10
    scores = cross_val_score(modelo, treino_dados, treino_marcacoes, cv = k)
    taxa_acerto = np.mean(scores)
    msg = 'Taxa de acerto do {0}: {1}'.format(nome, taxa_acerto)
    print(msg)
    return taxa_acerto

In [13]:
def teste_real(modelo, validacao_dados, validacao_marcacoes):
    resultado = modelo.predict(validacao_dados)
    
    acertos = resultado == validacao_marcacoes
    
    total_acertos = sum(acertos)
    total_elementos = len(validacao_marcacoes)
    
    taxa_acerto = 100.0 * total_acertos / total_elementos
    
    msg = "Taxa de acerto do vencedor entre os dois algoritmos no mundo real: {0}".format(taxa_acerto)
    
    print(msg)